In [1]:
import gpxpy
import gpxpy.gpx
import branca.colormap as cm
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from IPython.display import display
import tqdm
import os
from fit2gpx import Converter
import os

ModuleNotFoundError: No module named 'gpxpy'

In [ ]:
## CUSTOMIZATION

# Coordinates of the Initial position of the map ( In this case Berlin, Germany )
LOCATION = [52.3, 13.3]
ZOOM_START = 10
TILES = "OpenStreetMap"

# Custom Colormap to represent speeds
MY_COLORMAP = cm.LinearColormap(["blue", "green", "yellow", "orange", "red", "magenta"], vmin=5, vmax=50, index=[5, 15, 23, 30, 40, 50])

# Relative File directories from the location of this notebook
USE_FIT_FOLDER = True # if false fit_directory can be empty...
FIT_DIRECTORY = "Raddaten/fit_data/"
GPX_DIRECTORY = "Raddaten/gpx_data/"

In [ ]:
fit_data = os.listdir(FIT_DIRECTORY)
gpx_data = os.listdir(GPX_DIRECTORY)

if USE_FIT_FOLDER:
    conv = Converter()

    for file in tqdm.tqdm(fit_data):
        gpx_name = file.split(".")[0]+".gpx"
        if not gpx_name in gpx_data:
            conv.fit_to_gpx(FIT_DIRECTORY+file, GPX_DIRECTORY+gpx_name)

list_of_gpx_files = [GPX_DIRECTORY+filepath for filepath in os.listdir(GPX_DIRECTORY)]
"Found " + str(len(list_of_gpx_files)) + " gpx-files"

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 595/595 [00:00<00:00, 198631.88it/s]


'Found 595 gpx-files'

In [ ]:
def add_file_to_map(filename, map, min_length=20, opacity=0.3):

    with open(filename, "r") as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    if gpx.length_3d()/1000 < min_length:
        return

    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({"latitude": point.latitude, "longitude": point.longitude, "elevation": point.elevation})

    route_df = pd.DataFrame(route_info)
    
    poly_list = []
    for _, row in route_df.iterrows():
        poly_list.append((row["latitude"], row["longitude"]))

    month_dict = {"01": "Januar",
                  "02": "Februar",
                  "03": "März",
                  "04": "April",
                  "05": "Mai",
                  "06": "Juni",
                  "07": "Juli",
                  "08": "August",
                  "09": "September",
                  "10": "Oktober",
                  "11": "November",
                  "12": "Dezember"}

    # name = filename.split("/")[-1].split(".")[0]
    # date = name[6:8] +"."+ month_dict[name[4:6]] +""+ name[0:4]

    folium.PolyLine(poly_list, filename, opacity=opacity, color="blue").add_to(map)

def add_speed_file_to_map(filename, map, min_length=20, opacity=0.5):
    with open(filename, "r") as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    if gpx.length_3d()/1000 < min_length:
        return

    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            previous_point = None
            for idx, point in enumerate(segment.points):
                if idx != 0:
                    route_info.append({"latitude": point.latitude, "longitude": point.longitude, "elevation": point.elevation, "speed": point.speed_between(previous_point)})
                    previous_point = point
                else:
                    route_info.append({"latitude": point.latitude, "longitude": point.longitude, "elevation": point.elevation, "speed": 0})
                    previous_point = point 

    route_df = pd.DataFrame(route_info)

    route_df["speed"][route_df["speed"]>30] = 0

    poly_list = []
    speed = []
    for _, row in route_df.iterrows():
        poly_list.append((row["latitude"], row["longitude"]))
        speed.append(row["speed"]*3.6)

    folium.ColorLine(poly_list, speed, colormap=MY_COLORMAP, opacity=opacity, weight=5, popup=f"{gpx.length_3d()/1000} km \nin {gpx.get_duration()/3600} hours", tooltip=filename).add_to(speed_map)

def make_heat_map(filenames, map):

    gpx_tracks = []

    for file in filenames:
        with open(file, "r") as gpx_file:
            gpx_tracks.append(gpxpy.parse(gpx_file))

    route_info = []

    for track in gpx_tracks:
        for track2 in track.tracks:
            for segment in track2.segments:
                for point in segment.points:
                    route_info.append({"latitude": point.latitude, "longitude": point.longitude})

    route_df = pd.DataFrame(route_info)

    heat_data = [[row['latitude'],row['longitude']] for index, row in route_df.iterrows()]
    HeatMap(heat_data).add_to(map)

In [ ]:
heat_map = folium.Map(location=LOCATION, zoom_start=ZOOM_START, tiles=TILES)
route_map = folium.Map(location=LOCATION, zoom_start=ZOOM_START, tiles=TILES)
speed_map = folium.Map(location=LOCATION, zoom_start=ZOOM_START, tiles=TILES, prefer_canvas=True)

for file in tqdm.tqdm(list_of_gpx_files):
    add_file_to_map(file, route_map, min_length=50, opacity=1)
    add_speed_file_to_map(file, speed_map, min_length=50, opacity=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 595/595 [02:46<00:00,  3.57it/s]


In [2]:
display(route_map)

NameError: name 'route_map' is not defined

In [3]:
MY_COLORMAP.add_to(speed_map)

display(speed_map)


NameError: name 'MY_COLORMAP' is not defined

In [4]:
make_heat_map(list_of_gpx_files, heat_map)
display(heat_map)

NameError: name 'make_heat_map' is not defined